In [86]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import url_changes

from bs4 import BeautifulSoup

import pandas as pd
import time

##### Ouverture du site booking.com avec Selenium

In [87]:
driver = webdriver.Chrome()

driver.get("https://booking.com")

data = []

current_url = driver.current_url

##### Acceptation des cookies

In [88]:
accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    )
accept_button.click()

##### Recherche des hébergements sur Paris

In [89]:
search_box = WebDriverWait(driver, 10).until(
   EC.presence_of_element_located((By.NAME, "ss"))
)

search_box.send_keys("Paris")

search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//button[.//span[text()='Rechercher']]")
    )
)

search_button.click()

WebDriverWait(driver, 10).until(url_changes(current_url))

True

##### Fermeture de la pop-up

In [90]:
pop_up_element = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//div[@class='eb33ef7c47']"))
)

close_button = driver.find_element(By.CSS_SELECTOR, "div.abcc616ec7 button[aria-label='Ignorer les infos relatives à la connexion']")
close_button.click()

##### Affichage de tous les hébergements disponibles

In [28]:
buttonNotLoaded = True

while buttonNotLoaded:
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        load_more_button = driver.find_element(By.CSS_SELECTOR, "div[data-results-container='1'] > :last-child > button")
        print("Stop scrolling!")
        buttonNotLoaded = False
    except:
        print("Keep scrolling...")

while True:
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        load_more_button = driver.find_element(By.XPATH, "//button[.//span[text()='Afficher plus de résultats']]")

        load_more_button.click()
        time.sleep(1)
    except Exception as e:
        print("Erreur : ", e)
        break


Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Keep scrolling...
Stop scrolling!
Erreur :  Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[.//span[text()='Afficher plus de résultats']]"}
  (Session info: chrome=131.0.6778.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF766D42775+28773]
	(No symbol) [0x00007FF766CAAFB0]
	(No symbol) [0x00007FF766B4552A]
	(No symbol) [0x00007FF766B98EAE]
	(No symbol) [0x00007FF766B9919C]
	(No symbol) [0x00007FF766BE23F7]
	(No symbol) [0x00007FF766BBEFDF]
	(No symbol) [0x00007FF766BDF1A4]
	(No symbol) [0x00007FF766BBED43]
	(No symbol) [0x00007FF766B8A548

##### Récupération des hébergements

In [91]:
property_cards = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, "//div[@data-testid='property-card']"))
)

In [92]:
i=0

for card in property_cards:
    i+=1

i

25

Parcours de chaque hebergements et recuperations des notes par catégories

In [93]:
list_links = []
for card in property_cards:
    link = card.find_element(By.CSS_SELECTOR, "a[data-testid='title-link']").get_attribute("href")
    title = card.find_element(By.CSS_SELECTOR, "div[data-testid='title']").text
    arrondissement = card.find_element(By.CSS_SELECTOR, "span[data-testid='address']").text
    list_links.append([title, link, arrondissement])


for hotel_title, link, arrondissement in list_links:
    driver.get(link)
    
    time.sleep(1)


    try:
        # address_element = driver.find_elements(By.XPATH, "//div[contains(@class, 'a53cbfa6de f17adf7576')]")
        # adresse = address_element[0].text.split('\n')[0]
        adresse_element = driver.find_element(By.CSS_SELECTOR, "span.hp_address_subtitle")
        adresse = adresse_element.text.strip()
    except:
        adresse = ""
    
    try:
        description_elements = driver.find_elements(By.CSS_SELECTOR, "[data-testid='property-description']")
        description = " ".join([element.text for element in description_elements])
    except:
        description = ""
    
    try:
        points_fort_elements = driver.find_elements(By.CSS_SELECTOR, "[data-testid='property-most-popular-facilities-wrapper'] ul li")
        points_fort = [points_fort.text for points_fort in points_fort_elements]
    except:
        points_fort = ""

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    reviews = soup.find_all('div', {'data-testid': 'review-subscore'})
    
    notes = []

    for review in reviews:
        title = review.find('span', class_='be887614c2')
        score = review.find('div', class_='ccb65902b2 bdc1ea4a28')
        if title and score:
            title_text = title.text.strip()
            score_text = score.text.strip()
            notes.append({
                "title_text": title.text.strip(),
                "score_text": score.text.strip()
            })

    comments = soup.find_all('li', class_='b0932df2e7')
    hotel_comments = []
    for comment in comments:
        try:
            name = comment.find('div', class_='a3332d346a').text.strip()
            comment_text = comment.find('div', class_='a53cbfa6de').text.strip()
            hotel_comments.append({
                'Nom': name,
                'Commentaire': comment_text
            })
        except AttributeError as e:
            print(f"Erreur lors de l'extraction d'un commentaire pour {hotel_title}: {e}")

    data.append({
        "titre": hotel_title, 
        "lien_hotel": link,
        "arrondissement": arrondissement,
        "adresse": adresse,
        "description": description,
        "points_fort": points_fort,
        "notes": notes,
        "hotel_comments": hotel_comments
    })

Erreur lors de l'extraction d'un commentaire pour Angels Apartments Paris: 'NoneType' object has no attribute 'text'
Erreur lors de l'extraction d'un commentaire pour Angels Apartments Paris: 'NoneType' object has no attribute 'text'
Erreur lors de l'extraction d'un commentaire pour Angels Apartments Paris: 'NoneType' object has no attribute 'text'
Erreur lors de l'extraction d'un commentaire pour Angels Apartments Paris: 'NoneType' object has no attribute 'text'
Erreur lors de l'extraction d'un commentaire pour Angels Apartments Paris: 'NoneType' object has no attribute 'text'
Erreur lors de l'extraction d'un commentaire pour Angels Apartments Paris: 'NoneType' object has no attribute 'text'
Erreur lors de l'extraction d'un commentaire pour Sky of Paris: 'NoneType' object has no attribute 'text'
Erreur lors de l'extraction d'un commentaire pour Hotel Prince Albert Opéra: 'NoneType' object has no attribute 'text'
Erreur lors de l'extraction d'un commentaire pour Hotel Prince Albert Opé

##### Fermeture de la page avec Selenium

In [128]:
driver.quit()

##### Création du dataframe et ajout au fichier csv

In [94]:
df = pd.DataFrame(data, columns=["titre", "lien_hotel","arrondissement","adresse","description","points_fort","notes", "hotel_comments"])
df.to_csv("booking.csv", index=False)

In [95]:
df

,titre,lien_hotel,arrondissement,adresse,description,points_fort,notes,hotel_comments
0,Angels Apartments Paris,https://www.booking.com/hotel/fr/magnifique-pe...,"19e arr., Paris",,"Situé à Paris, à 1,7 km de la gare du Nord et ...","[Piscine intérieure, Parking gratuit, Connexio...","[{'title_text': 'Personnel', 'score_text': '10...","[{'Nom': 'Tai', 'Commentaire': '« Thank you fo..."
1,Sky of Paris,https://www.booking.com/hotel/fr/sky-of-paris....,"7e arr., Paris",,"Situé dans le centre de Paris, à 700 mètres du...","[Connexion Wi-Fi gratuite, Ascenseur, Chauffag...","[{'title_text': 'Personnel', 'score_text': '10...","[{'Nom': 'Anna', 'Commentaire': '« J’ai passé ..."
2,Hotel Prince Albert Opéra,https://www.booking.com/hotel/fr/prince-albert...,"9e arr., Paris",,"Situé dans le centre de Paris, à 2 minutes à p...","[Connexion Wi-Fi gratuite, Chambres familiales...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Sébastien', 'Commentaire': '« La cha..."
3,Cosy 1 bedroom Paris 15,https://www.booking.com/hotel/fr/5-min-from-va...,"15e arr., Paris",,"Situé à Paris, en Île-de-France, le Cosy 1 bed...","[Connexion Wi-Fi gratuite, Chambres non-fumeur...","[{'title_text': 'Personnel', 'score_text': '10...","[{'Nom': 'Alexandra', 'Commentaire': '« L empl..."
4,appartement atypique aux Batignolles,https://www.booking.com/hotel/fr/appartement-a...,"17e arr., Paris",,"Offrant une vue sur la cour intérieure, l'appa...","[Connexion Wi-Fi gratuite, Chambres non-fumeur...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Anne', 'Commentaire': '« Studio ""cos..."
5,Superb T2 avec balcon 10mn de la Gare Montparn...,https://www.booking.com/hotel/fr/appartement-l...,"15e arr., Paris",,"Set in Paris in the Ile de France region, Supe...","[Connexion Wi-Fi gratuite, Connexion Wi-Fi gra...","[{'title_text': 'Personnel', 'score_text': '8,...","[{'Nom': 'Maria', 'Commentaire': '« El diseño ..."
6,Kyriad Hotel XIII Italie Gobelins,https://www.booking.com/hotel/fr/hotelkyriadit...,"13e arr., Paris",,Le Kyriad Hotel XIII Italie Gobelins est situé...,"[Connexion Wi-Fi gratuite, Chambres familiales...","[{'title_text': 'Personnel', 'score_text': '8,...","[{'Nom': 'Frederic', 'Commentaire': '« personn..."
7,Studio Bourdonnais,https://www.booking.com/hotel/fr/studio-bourdo...,"7e arr., Paris",,"Situé à Paris, le Studio Bourdonnais est à 500...","[Connexion Wi-Fi gratuite, Chambres non-fumeur...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Michelle', 'Commentaire': '« The Stu..."
8,Le Trésor,https://www.booking.com/hotel/fr/le-tresor-par...,"8e arr., Paris",,"Idéalement situé dans le centre de Paris, l'ét...","[Parking, Connexion Wi-Fi gratuite, Réception ...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Frédéric', 'Commentaire': '« Logemen..."
9,Maison de ville cosy Canal Saint Martin,https://www.booking.com/hotel/fr/nid-douillet-...,"11e arr., Paris",,La Maison de ville cosy Canal Saint Martin est...,"[Parking, Connexion Wi-Fi gratuite, Chambres n...","[{'title_text': 'Personnel', 'score_text': '9,...","[{'Nom': 'Avrin', 'Commentaire': '« Très bon a..."


##### Ajout à la base de données (SQLite & MongoDB)

In [131]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["booking_db"]
collection = db["hebergements"]

collection.insert_many(df.to_dict(orient='records'))

InsertManyResult([ObjectId('673a852bf9d997c8b1499c59'), ObjectId('673a852bf9d997c8b1499c5a'), ObjectId('673a852bf9d997c8b1499c5b'), ObjectId('673a852bf9d997c8b1499c5c'), ObjectId('673a852bf9d997c8b1499c5d'), ObjectId('673a852bf9d997c8b1499c5e'), ObjectId('673a852bf9d997c8b1499c5f'), ObjectId('673a852bf9d997c8b1499c60'), ObjectId('673a852bf9d997c8b1499c61'), ObjectId('673a852bf9d997c8b1499c62'), ObjectId('673a852bf9d997c8b1499c63'), ObjectId('673a852bf9d997c8b1499c64'), ObjectId('673a852bf9d997c8b1499c65'), ObjectId('673a852bf9d997c8b1499c66'), ObjectId('673a852bf9d997c8b1499c67'), ObjectId('673a852bf9d997c8b1499c68'), ObjectId('673a852bf9d997c8b1499c69'), ObjectId('673a852bf9d997c8b1499c6a'), ObjectId('673a852bf9d997c8b1499c6b'), ObjectId('673a852bf9d997c8b1499c6c'), ObjectId('673a852bf9d997c8b1499c6d'), ObjectId('673a852bf9d997c8b1499c6e'), ObjectId('673a852bf9d997c8b1499c6f'), ObjectId('673a852bf9d997c8b1499c70'), ObjectId('673a852bf9d997c8b1499c71'), ObjectId('673a852bf9d997c8b1499c